In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [2]:
cd ..

C:\Users\noahc\Value_Repo


In [3]:
df_dict= pd.read_excel("data//raw//stock_dataXL.xlsx",sheet_name= None)

In [4]:
df_dict_price = df_dict.copy()

In [5]:
for key in df_dict_price:
    df_dict_price[key] = df_dict_price[key][['Date','Price']]
for key in df_dict_price:
    print(df_dict_price[key].head())

                  Date   Price
0  2021-03-31 00:00:00  291.02
1  2021-03-30 00:00:00  293.25
2  2021-03-29 00:00:00   293.3
3  2021-03-26 00:00:00  291.22
4  2021-03-25 00:00:00  286.66
                  Date  Price
0  2021-03-31 00:00:00  19.41
1  2021-03-30 00:00:00   19.5
2  2021-03-29 00:00:00   19.1
3  2021-03-26 00:00:00  19.25
4  2021-03-25 00:00:00  20.02
                  Date Price
0  2021-03-31 00:00:00  3.75
1  2021-03-30 00:00:00  3.73
2  2021-03-29 00:00:00   3.7
3  2021-03-28 00:00:00  3.71
4  2021-03-25 00:00:00  3.71
                  Date  Price
0  2021-03-31 00:00:00   10.2
1  2021-03-30 00:00:00  10.36
2  2021-03-29 00:00:00  10.47
3  2021-03-26 00:00:00   9.83
4  2021-03-25 00:00:00    9.8
                  Date  Price
0  2021-03-31 00:00:00  85.15
1  2021-03-30 00:00:00   86.1
2  2021-03-29 00:00:00     85
3  2021-03-26 00:00:00  81.85
4  2021-03-25 00:00:00   83.6
                  Date Price
0  2021-03-31 00:00:00  5150
1  2021-03-30 00:00:00  5100
2  2021-03-29

In [6]:
df = df_dict_price['Russia - Sberbank Rossii PAO (S']

In [7]:
df = df[:-1]

In [8]:
# Extract the 'Date' column and use it as the index
df.loc[:, 'Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

C:\Users\noahc\AppData\Local\Temp\ipykernel_13972\4011687541.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Date'] = pd.to_datetime(df['Date'])
C:\Users\noahc\AppData\Local\Temp\ipykernel_13972\4011687541.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'Date'] = pd.to_datetime(df['Date'])


In [9]:
df.head()

,Price
Date,
2021-03-31,291.02
2021-03-30,293.25
2021-03-29,293.3
2021-03-26,291.22
2021-03-25,286.66


In [10]:
#df['Vol.'] = df['Vol.'].str.replace('M', '').astype(float)

In [11]:
# Split the data into training and test sets
train_data = df[df.index.year != 2021]
test_data = df[df.index.year == 2021]

In [12]:
# Normalize the data using MinMaxScaler
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [13]:
# Convert the normalized training data into a 3D array
X_train = []
y_train = []
for i in range(60, train_data.shape[0]):
    X_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [14]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [15]:
# Train the LSTM model on the training data
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
6/6 [==============================] - 4s 11ms/step - loss: 0.0851
Epoch 2/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0289
Epoch 3/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0156
Epoch 4/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0169
Epoch 5/100
6/6 [==============================] - 0s 10ms/step - loss: 0.0129
Epoch 6/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0118
Epoch 7/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0113
Epoch 8/100
6/6 [==============================] - 0s 10ms/step - loss: 0.0102
Epoch 9/100
6/6 [==============================] - 0s 10ms/step - loss: 0.0096
Epoch 10/100
6/6 [==============================] - 0s 11ms/step - loss: 0.0092
Epoch 11/100
6/6 [==============================] - 0s 10ms/step - loss: 0.0085
Epoch 12/100
6/6 [==============================] - 0s 10ms/step - loss: 0.0081
Epoch 13/100
6/6 [==============================]

In [16]:
# Use the trained model to make predictions on the test data
test_data = scaler.transform(test_data)
X_test = []
for i in range(60, test_data.shape[0]):
    X_test.append(test_data[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:409: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [18]:
# Generate predictions for all 61 time steps in the test data
predictions = []
for i in range(60, 121):
    x = test_data[i-60:i, :]
    x = x.reshape(1, 60, 1)
    yhat = model.predict(x, verbose=0)
    predictions.append(yhat[0,0])

ValueError: cannot reshape array of size 59 into shape (1,60,1)

In [ ]:
# Calculate the BUY, HOLD, and SELL recommendations based on the predictions
recommendations = []
for i in range(len(predictions)):
    if predictions[i] > test_data[60+i, 0]:
        recommendations.append('BUY')
    elif predictions[i] < test_data[60+i, 0]:
        recommendations.append('SELL')
    else:
        recommendations.append('HOLD')

In [ ]:
# Calculate the capital gains/losses for each BUY, HOLD, and SELL event
gains_losses = []
for i in range(len(recommendations)):
    if recommendations[i] == 'BUY':
        gains_losses.append(test_data[60+i, 0] - test_data[59+i, 0])
    elif recommendations[i] == 'SELL':
        gains_losses.append(test_data[59+i, 0] - test_data[60+i, 0])
    else:
        gains_losses.append(0)


In [ ]:
# Aggregate the capital gains/losses into a total
total = np.sum(gains_losses)


In [ ]:
# Store the actual prices and the predicted prices in the same DataFrame
results = df[df.index >= '2021-01-01']
results['Predicted'] = predictions

# Plot the actual stock prices and the predictions
plt.plot(results['Price'], label='Actual')
plt.plot(results['Predicted'], label='Predicted')
plt.legend()
plt.show()

In [ ]:
print('Recommendations:', recommendations)
print('Capital gains/losses:', gains_losses)
print('Total:', total)